# DMD++ Fingerprint Identification with Background Gallery

This notebook performs 1:N fingerprint identification using DMD++ templates.

**Experiment Setup:**
- **Query Set:** 258 latent fingerprints from SD258 dataset (sid=0)
- **Gallery Set:** 258 reference fingerprints from SD258 (sid=1) [+ background from TS1K (sid=0), optional]
- **Task:** Identify each query in the large gallery

**Checkpoints:** Templates and identification results are saved to disk to avoid recomputation.

## 0. Configuration

Set all experiment parameters here.

In [ ]:
# ========== EXPERIMENT CONFIGURATION ==========

# Number of background samples to use from TS1K (set to None to use all available)
NUM_BACKGROUND_SAMPLES = 0  # Use None for all, or set a number like 1000, 5000, 10000

# Template extraction settings
BATCH_SIZE = 64
USE_GPU = True
DEVICE = 'cuda:4' if USE_GPU else 'cpu'

# Identification settings
IDENTIFICATION_BATCH_SIZE = 256

# Results folder
RESULTS_FOLDER = "dmd_sd27"

# Error visualization
MAX_DISPLAY_ERR = 10

# CMC settings
MAX_CMC_RANK = 258  # Maximum rank to compute for CMC curve

print("Configuration loaded successfully!")
print(f"  Device: {DEVICE}")
print(f"  Background samples: {NUM_BACKGROUND_SAMPLES if NUM_BACKGROUND_SAMPLES else 'All available'}")
print(f"  Results folder: {RESULTS_FOLDER}")

## 1. Imports and Setup

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pickle as pkl
import os
from glob import glob
from tqdm.notebook import tqdm
import torch

import dmd
import grids as gr

In [ ]:
# Create folder structure
os.makedirs(RESULTS_FOLDER, exist_ok=True)

# Query folder (SD258 latents)
query_folder = os.path.join(RESULTS_FOLDER, "query")
os.makedirs(query_folder, exist_ok=True)

# Gallery folders
gallery_sd258_folder = os.path.join(RESULTS_FOLDER, "gallery_sd258")
os.makedirs(gallery_sd258_folder, exist_ok=True)

gallery_ts1k_folder = os.path.join(RESULTS_FOLDER, "gallery_ts1k")
os.makedirs(gallery_ts1k_folder, exist_ok=True)

print(f"Folder structure created:")
print(f"  Query: {query_folder}")
print(f"  Gallery SD258: {gallery_sd258_folder}")
print(f"  Gallery TS1K: {gallery_ts1k_folder}")

# Helper function
noext = lambda f: os.path.splitext(f)[0]

## 2. Load Datasets

In [ ]:
# Load SD258 dataset
print("Loading SD258 dataset...")
ds_sd258 = gr.datasets.SD258
ds_sd258.load(layers=["orig"])

# Query set: SD258 latents (sid=0)
queries = ds_sd258[{"sid": 0}]
print(f"  Queries (SD258 sid=0): {len(queries)} samples")

# Gallery SD258: references (sid=1)
gallery_sd258 = ds_sd258[{"sid": 1}]
print(f"  Gallery SD258 (sid=1): {len(gallery_sd258)} samples")

# Load TS1K dataset for background gallery only if needed
if NUM_BACKGROUND_SAMPLES is None or NUM_BACKGROUND_SAMPLES > 0:
    print("\nLoading TS1K dataset for background...")
    ds_ts1k = gr.datasets.TS1K
    ds_ts1k.log()
    ds_ts1k.load(layers=["orig"])

    # Background gallery: TS1K (sid=0 only, to avoid duplicates)
    gallery_ts1k_full = ds_ts1k[{"sid": 0}]
    print(f"  Total TS1K samples available (sid=0): {len(gallery_ts1k_full)}")

    # Apply limit if specified
    if NUM_BACKGROUND_SAMPLES is not None:
        gallery_ts1k = [gallery_ts1k_full[i] for i in range(min(NUM_BACKGROUND_SAMPLES, len(gallery_ts1k_full)))]
        print(f"  Using limited TS1K samples: {len(gallery_ts1k)}")
    else:
        gallery_ts1k = gallery_ts1k_full
        print(f"  Using all available TS1K samples: {len(gallery_ts1k)}")
else:
    print("\nSkipping TS1K dataset (NUM_BACKGROUND_SAMPLES = 0)")
    gallery_ts1k = []

print(f"\nFinal configuration:")
print(f"  Queries: {len(queries)}")
print(f"  Gallery (SD258): {len(gallery_sd258)}")
print(f"  Gallery (TS1K): {len(gallery_ts1k)}")
print(f"  Total Gallery: {len(gallery_sd258) + len(gallery_ts1k)}")

## 3. Template Extraction

Extract DMD++ templates for all fingerprints. Templates are saved to disk to avoid recomputation.

In [ ]:
# Check if templates already exist
query_files = sorted(glob(os.path.join(query_folder, "*.pkl")))
gallery_sd258_files = sorted(glob(os.path.join(gallery_sd258_folder, "*.pkl")))
gallery_ts1k_files = sorted(glob(os.path.join(gallery_ts1k_folder, "*.pkl")))

# Determine expected number of TS1K templates based on NUM_BACKGROUND_SAMPLES
if NUM_BACKGROUND_SAMPLES is None or NUM_BACKGROUND_SAMPLES > 0:
    expected_ts1k_templates = len(gallery_ts1k)
else:
    expected_ts1k_templates = 0

templates_exist = (
    len(query_files) == len(queries) and
    len(gallery_sd258_files) == len(gallery_sd258) and
    len(gallery_ts1k_files) >= expected_ts1k_templates
)



if templates_exist:
    print("✅ Templates already extracted!")
    print(f"   Queries: {len(query_files)} files")
    print(f"   Gallery SD258: {len(gallery_sd258_files)} files")
    print(f"   Gallery TS1K: {len(gallery_ts1k_files)} files (need {expected_ts1k_templates})")
    print("\n   Skipping extraction. Delete template folders to re-extract.")
else:
    print("🔧 Templates not found or incomplete. Starting extraction...\n")
    
    # Helper function to move template to CPU
    def move_template_to_cpu(template):
        """Move all tensors in a template to CPU for storage."""
        template_cpu = {}
        for key, value in template.items():
            if isinstance(value, torch.Tensor):
                template_cpu[key] = value.cpu()
            else:
                template_cpu[key] = value
        return template_cpu
    
    # Create DMD pipeline
    mntstitch_model_path = "/fast8TB/jcontreras/work/grfinger/patchgrid/training_template_stitch/notebooks/mntstitch/models/resnet34_v2.pt"

    # Helper function to extract templates
    def extract_templates(dataset, folder, desc):
        """Extract DMD templates for a dataset and save to folder."""
        template_list = []
        
        for item in tqdm(dataset, desc=desc):
            # Extract template
            out_dmd = dmd_pipe.run({"raw_image": item["orig"]})
            template = out_dmd["dmd"]
            
            # Move to CPU before saving (for portability)
            template_cpu = move_template_to_cpu(template)
            template_list.append(template)
            
            # Save to disk (CPU version for portability)
            template_path = os.path.join(folder, f"{noext(item.external_id)}.pkl")
            with open(template_path, "wb") as f:
                pkl.dump(template_cpu, f)
        
        return template_list
    
    # Query extraction pipeline
    dmd_pipe = gr.pipelines.dmd(
        mnt_extractor="mntstitch",
        mntstitch_model_path=mntstitch_model_path,
        device=DEVICE,
        use_gpu_patches=USE_GPU,
        max_batch_size=BATCH_SIZE,
        mntstitch_kwargs={"qthreshold": 0.15}
    )

    # Extract query templates
    print(f"📦 Extracting {len(queries)} query templates...")
    query_templates = extract_templates(queries, query_folder, "Queries")
    
    # Gallery extraction pipeline
    dmd_pipe = gr.pipelines.dmd(
        mnt_extractor="mntstitch",
        mntstitch_model_path=mntstitch_model_path,
        device=DEVICE,
        use_gpu_patches=USE_GPU,
        max_batch_size=BATCH_SIZE
    )
    

    # Extract SD258 gallery templates
    print(f"\n📦 Extracting {len(gallery_sd258)} SD258 gallery templates...")
    gallery_sd258_templates = extract_templates(gallery_sd258, gallery_sd258_folder, "Gallery SD258")
    
    # Extract TS1K gallery templates only if needed
    if len(gallery_ts1k) > 0:
        print(f"\n Extracting {len(gallery_ts1k)} TS1K gallery templates...")
        gallery_ts1k_templates = extract_templates(gallery_ts1k, gallery_ts1k_folder, "Gallery TS1K")
    else:
        print(f"\n Skipping TS1K template extraction (NUM_BACKGROUND_SAMPLES = 0)")
        gallery_ts1k_templates = []
    
    print(f"\n✅ Template extraction complete!")
    print(f"   Total templates: {len(query_templates) + len(gallery_sd258_templates) + len(gallery_ts1k_templates)}")
    print(f"   Templates saved to CPU for portability")

## 4. Load Templates from Disk

In [ ]:
print("Loading templates from disk...\n")

def move_template_to_device(template, device):
    """Move all tensors in a template to the specified device."""
    template_moved = {}
    for key, value in template.items():
        if isinstance(value, torch.Tensor):
            template_moved[key] = value.to(device)
        else:
            template_moved[key] = value
    return template_moved

# Load query templates
query_files = sorted(glob(os.path.join(query_folder, "*.pkl")))
query_templates = []
for f in tqdm(query_files, desc="Loading queries"):
    with open(f, "rb") as pkl_file:
        template = pkl.load(pkl_file)
        # Move template to configured device
        template = move_template_to_device(template, DEVICE)
        query_templates.append(template)

# Load SD258 gallery templates
gallery_sd258_files = sorted(glob(os.path.join(gallery_sd258_folder, "*.pkl")))
gallery_sd258_templates = []
for f in tqdm(gallery_sd258_files, desc="Loading SD258 gallery"):
    with open(f, "rb") as pkl_file:
        template = pkl.load(pkl_file)
        # Move template to configured device
        template = move_template_to_device(template, DEVICE)
        gallery_sd258_templates.append(template)

# Load TS1K gallery templates only if needed
if NUM_BACKGROUND_SAMPLES is None or NUM_BACKGROUND_SAMPLES > 0:
    gallery_ts1k_files = sorted(glob(os.path.join(gallery_ts1k_folder, "*.pkl")))
    gallery_ts1k_files = gallery_ts1k_files[:NUM_BACKGROUND_SAMPLES if NUM_BACKGROUND_SAMPLES else None]
    gallery_ts1k_templates = []
    for f in tqdm(gallery_ts1k_files, desc="Loading TS1K gallery"):
        with open(f, "rb") as pkl_file:
            template = pkl.load(pkl_file)
            # Move template to configured device
            template = move_template_to_device(template, DEVICE)
            gallery_ts1k_templates.append(template)
else:
    print("Skipping TS1K template loading (NUM_BACKGROUND_SAMPLES = 0)")
    gallery_ts1k_files = []
    gallery_ts1k_templates = []

# Combine gallery templates (SD258 first, then TS1K)
gallery_templates = gallery_sd258_templates + gallery_ts1k_templates

print(f"\n✅ Templates loaded and moved to {DEVICE}:")
print(f"   Queries: {len(query_templates)}")
print(f"   Gallery SD258: {len(gallery_sd258_templates)}")
print(f"   Gallery TS1K: {len(gallery_ts1k_templates)}")
print(f"   Total Gallery: {len(gallery_templates)}")

## 4.1. Template Quality Check

Identify and remove templates with no minutiae (which cause matching errors).

In [ ]:
import torch
import os

def check_template_quality(templates, names, filter_low_minutiae=True):
    """
    Check template quality and identify problematic templates.
    
    Args:
        filter_low_minutiae: If True, filter templates with < 4 minutiae (for gallery only)
    
    Returns:
        bad_indices: List of indices with problems
        reasons: Dict mapping index to reason for failure
    """
    bad_indices = []
    reasons = {}
    
    for i, template in enumerate(templates):
        problems = []
        
        # Check minutiae
        mnt = template['mnt']
        num_mnt = 0
        
        if isinstance(mnt, torch.Tensor):
            if mnt.shape[0] == 0 or (len(mnt.shape) > 1 and mnt.shape[1] == 0):
                problems.append("empty_mnt_shape")
            else:
                # Count valid minutiae (non-NaN in first column)
                if mnt.numel() > 0:
                    # Handle different shapes: (N, 3) or (1, N, 3)
                    if len(mnt.shape) == 3:
                        num_mnt = torch.sum(~torch.isnan(mnt[0, :, 0])).item()
                    else:
                        num_mnt = torch.sum(~torch.isnan(mnt[:, 0])).item()
                
                if num_mnt == 0:
                    problems.append("zero_minutiae")
                elif filter_low_minutiae and num_mnt < 4:
                    problems.append(f"too_few_minutiae({num_mnt})")
        else:
            # NumPy array
            if mnt.shape[0] == 0 or (len(mnt.shape) > 1 and mnt.shape[1] == 0):
                problems.append("empty_mnt_shape")
            else:
                if mnt.size > 0:
                    # Handle different shapes
                    if len(mnt.shape) == 3:
                        num_mnt = np.sum(~np.isnan(mnt[0, :, 0]))
                    else:
                        num_mnt = np.sum(~np.isnan(mnt[:, 0]))
                
                if num_mnt == 0:
                    problems.append("zero_minutiae")
                elif filter_low_minutiae and num_mnt < 4:
                    problems.append(f"too_few_minutiae({num_mnt})")
        
        # Check features (DMD uses 'feature' key, not 'desc')
        feat = template['feature']
        if isinstance(feat, torch.Tensor):
            if feat.numel() == 0:
                problems.append("empty_feature")
            elif torch.isnan(feat).any():
                problems.append("nan_features")
            elif torch.isinf(feat).any():
                problems.append("inf_features")
        else:
            # NumPy array
            if feat.size == 0:
                problems.append("empty_feature")
            elif np.isnan(feat).any():
                problems.append("nan_features")
            elif np.isinf(feat).any():
                problems.append("inf_features")
        
        if problems:
            bad_indices.append(i)
            reasons[i] = problems
    
    return bad_indices, reasons

# Check quality of all template sets
print("🔍 Checking template quality...\n")

# For queries (latents): only check for reporting, DO NOT filter
bad_queries, reasons_q = check_template_quality(query_templates, "Queries", filter_low_minutiae=False)

# For gallery (references): check everything including low minutiae count
bad_sd258, reasons_sd258 = check_template_quality(gallery_sd258_templates, "Gallery SD258", filter_low_minutiae=True)
bad_ts1k, reasons_ts1k = check_template_quality(gallery_ts1k_templates, "Gallery TS1K", filter_low_minutiae=True)

print(f"Problematic templates found:")
print(f"  Queries (latents): {len(bad_queries)} / {len(query_templates)} [reporting only, NOT filtered]")
if bad_queries and len(bad_queries) <= 10:
    for idx in bad_queries:
        print(f"    [{idx}] {reasons_q[idx]}")

print(f"  Gallery SD258 (references): {len(bad_sd258)} / {len(gallery_sd258_templates)} [will be filtered]")
if bad_sd258 and len(bad_sd258) <= 10:
    for idx in bad_sd258:
        print(f"    [{idx}] {reasons_sd258[idx]}")

print(f"  Gallery TS1K (references): {len(bad_ts1k)} / {len(gallery_ts1k_templates)} [will be filtered]")
if bad_ts1k:
    if len(bad_ts1k) <= 20:
        for idx in bad_ts1k:
            ts1k_id = noext(os.path.basename(gallery_ts1k_files[idx]))
            print(f"    [{idx}] {ts1k_id}: {reasons_ts1k[idx]}")
    else:
        print(f"    Too many bad templates ({len(bad_ts1k)}), showing first 20:")
        for idx in bad_ts1k[:20]:
            ts1k_id = noext(os.path.basename(gallery_ts1k_files[idx]))
            print(f"    [{idx}] {ts1k_id}: {reasons_ts1k[idx]}")
        print(f"    ... and {len(bad_ts1k) - 20} more")

# Filter ONLY gallery templates, NEVER queries
if len(bad_sd258) > 0 or len(bad_ts1k) > 0:
    print(f"\n⚠️  Found {len(bad_sd258) + len(bad_ts1k)} problematic gallery templates!")
    print(f"   Filtering them out...\n")
    
    # NEVER filter queries - keep all of them
    query_templates_filtered = query_templates
    queries_filtered = queries
    query_ids_filtered = [noext(os.path.basename(f)) for f in query_files]
    
    # Filter SD258 gallery
    if len(bad_sd258) > 0:
        gallery_sd258_templates_filtered = [t for i, t in enumerate(gallery_sd258_templates) if i not in bad_sd258]
        gallery_sd258_filtered = [g for i, g in enumerate(gallery_sd258) if i not in bad_sd258]
        gallery_sd258_ids_filtered = [noext(os.path.basename(gallery_sd258_files[i])) for i in range(len(gallery_sd258_files)) if i not in bad_sd258]
        print(f"   Removed {len(bad_sd258)} SD258 gallery. Remaining: {len(gallery_sd258_templates_filtered)}")
    else:
        gallery_sd258_templates_filtered = gallery_sd258_templates
        gallery_sd258_filtered = gallery_sd258
        gallery_sd258_ids_filtered = [noext(os.path.basename(f)) for f in gallery_sd258_files]
    
    # Filter TS1K gallery
    if len(bad_ts1k) > 0:
        gallery_ts1k_templates_filtered = [t for i, t in enumerate(gallery_ts1k_templates) if i not in bad_ts1k]
        gallery_ts1k_filtered = [g for i, g in enumerate(gallery_ts1k) if i not in bad_ts1k]
        gallery_ts1k_ids_filtered = [noext(os.path.basename(gallery_ts1k_files[i])) for i in range(len(gallery_ts1k_files)) if i not in bad_ts1k]
        print(f"   Removed {len(bad_ts1k)} TS1K gallery. Remaining: {len(gallery_ts1k_templates_filtered)}")
    else:
        gallery_ts1k_templates_filtered = gallery_ts1k_templates
        gallery_ts1k_filtered = gallery_ts1k
        gallery_ts1k_ids_filtered = [noext(os.path.basename(f)) for f in gallery_ts1k_files]
    
    # Update references
    query_templates = query_templates_filtered
    queries = queries_filtered
    query_ids = query_ids_filtered
    
    gallery_sd258_templates = gallery_sd258_templates_filtered
    gallery_sd258 = gallery_sd258_filtered
    gallery_sd258_ids = gallery_sd258_ids_filtered
    
    gallery_ts1k_templates = gallery_ts1k_templates_filtered
    gallery_ts1k = gallery_ts1k_filtered
    gallery_ts1k_ids = gallery_ts1k_ids_filtered
    
    # Recombine gallery
    gallery_templates = gallery_sd258_templates + gallery_ts1k_templates
    gallery_ids = gallery_sd258_ids + gallery_ts1k_ids
    
    print(f"\n✅ Filtered templates:")
    print(f"   Queries: {len(query_templates)} (no filtering applied)")
    print(f"   Gallery SD258: {len(gallery_sd258_templates)}")
    print(f"   Gallery TS1K: {len(gallery_ts1k_templates)}")
    print(f"   Total Gallery: {len(gallery_templates)}")
else:
    print(f"\n✅ All gallery templates have valid quality. No filtering needed.")
    
    # Still need to create ID lists for ground truth mapping
    query_ids = [noext(os.path.basename(f)) for f in query_files]
    gallery_sd258_ids = [noext(os.path.basename(f)) for f in gallery_sd258_files]
    gallery_ts1k_ids = [noext(os.path.basename(f)) for f in gallery_ts1k_files]
    gallery_ids = gallery_sd258_ids + gallery_ts1k_ids

## 5. Identification (1:N Matching)

Compute similarity scores between all queries and gallery templates.
Results are saved to avoid recomputation.

In [ ]:
# Define path for identification results
identification_results_path = os.path.join(RESULTS_FOLDER, "identification_results.pkl")

# Check if results already exist
if os.path.exists(identification_results_path):
    print("📂 Loading identification results from disk...")
    with open(identification_results_path, "rb") as f:
        results = pkl.load(f)
        scores_matrix = results["scores_matrix"]
    
    print(f"✅ Loaded scores matrix!")
    print(f"   Score matrix shape: {scores_matrix.shape}")
    print(f"   Score range: [{np.nanmin(scores_matrix):.4f}, {np.nanmax(scores_matrix):.4f}]")
    
    # ALWAYS rebuild ground truth with current IDs (in case filtering changed)
    print(f"\n📋 Rebuilding ground truth matrix with current IDs...")
    
    num_queries = len(query_templates)
    num_gallery = len(gallery_templates)
    num_sd258_gallery = len(gallery_sd258_templates)
    
    target_matrix = np.zeros((num_queries, num_gallery), dtype=bool)
    
    # For each query, find its matching gallery image in SD258 part
    matches_found = 0
    matches_not_found = []
    
    for q_idx, q_id in enumerate(query_ids):
        # Extract prefix (e.g., "sd258_000_11-00_latent_bad" -> "sd258_000_11")
        q_prefix = "_".join(q_id.split("_")[:2])
        
        # Find matching gallery ID in SD258 part only
        found = False
        for g_idx, g_id in enumerate(gallery_sd258_ids):
            g_prefix = "_".join(g_id.split("_")[:2])
            if q_prefix == g_prefix:
                # This is the correct match
                target_matrix[q_idx, g_idx] = True
                matches_found += 1
                found = True
                break
        
        if not found:
            matches_not_found.append((q_idx, q_id))
    
    print(f"   Matches found: {matches_found} / {num_queries}")
    
    if matches_not_found:
        print(f"   ⚠️  {len(matches_not_found)} queries without matches:")
        for q_idx, q_id in matches_not_found[:10]:
            print(f"      [{q_idx}] {q_id}")
        if len(matches_not_found) > 10:
            print(f"      ... and {len(matches_not_found) - 10} more")
    
    print(f"\n📈 Ground truth matrix:")
    print(f"   Shape: {target_matrix.shape}")
    print(f"   Genuine pairs (SD258): {target_matrix.sum()}")
    print(f"   Impostor pairs (TS1K): {num_queries * len(gallery_ts1k_templates)}")
    
    # Save updated results
    print(f"\n💾 Saving updated results...")
    results = {
        "scores_matrix": scores_matrix,
        "target_matrix": target_matrix,
        "query_ids": query_ids,
        "gallery_ids": gallery_ids,
        "metadata": {
            "num_queries": num_queries,
            "num_gallery_sd258": num_sd258_gallery,
            "num_gallery_ts1k": len(gallery_ts1k_templates),
            "num_gallery_total": num_gallery,
            "matches_found": matches_found
        }
    }
    with open(identification_results_path, "wb") as f:
        pkl.dump(results, f)
    print(f"   Saved to: {identification_results_path}")

else:
    print("🚀 Starting 1:N identification...")
    print(f"   Computing {len(query_templates)} × {len(gallery_templates)} = {len(query_templates) * len(gallery_templates):,} comparisons")
    print(f"   Device: {DEVICE}")
    print(f"   Batch size: {IDENTIFICATION_BATCH_SIZE}")
    
    # Create matcher
    matcher = dmd.DmdMatcher()
    
    # Perform identification
    scores_matrix = matcher.identify(
        query_templates,
        gallery_templates,
        device=DEVICE,
        batch_size=IDENTIFICATION_BATCH_SIZE
    )
    
    print(f"\n✅ Identification complete!")
    print(f"   Score matrix shape: {scores_matrix.shape}")
    print(f"   Score range: [{np.nanmin(scores_matrix):.4f}, {np.nanmax(scores_matrix):.4f}]")
    
    # Create ground truth matrix by matching IDs
    # Query IDs are like "sd258_000_11-00_latent_bad", Gallery IDs are like "sd258_000_11-01_template_bad"
    # They match if the prefix (first 2 parts) is the same
    num_queries = len(query_templates)
    num_gallery = len(gallery_templates)
    num_sd258_gallery = len(gallery_sd258_templates)
    
    target_matrix = np.zeros((num_queries, num_gallery), dtype=bool)
    
    print(f"\n📋 Building ground truth matrix by matching IDs...")
    
    # For each query, find its matching gallery image in SD258 part
    matches_found = 0
    matches_not_found = []
    
    for q_idx, q_id in enumerate(query_ids):
        # Extract prefix (e.g., "sd258_000_11-00_latent_bad" -> "sd258_000_11")
        q_prefix = "_".join(q_id.split("_")[:2])
        
        # Find matching gallery ID in SD258 part only
        found = False
        for g_idx, g_id in enumerate(gallery_sd258_ids):
            g_prefix = "_".join(g_id.split("_")[:2])
            if q_prefix == g_prefix:
                # This is the correct match
                target_matrix[q_idx, g_idx] = True
                matches_found += 1
                found = True
                break
        
        if not found:
            matches_not_found.append((q_idx, q_id))
    
    print(f"   Matches found: {matches_found} / {num_queries}")
    
    if matches_not_found:
        print(f"   ⚠️  {len(matches_not_found)} queries without matches:")
        for q_idx, q_id in matches_not_found[:10]:
            print(f"      [{q_idx}] {q_id}")
        if len(matches_not_found) > 10:
            print(f"      ... and {len(matches_not_found) - 10} more")
    
    # TS1K entries are all impostors (already False)
    
    print(f"\n📈 Ground truth matrix:")
    print(f"   Shape: {target_matrix.shape}")
    print(f"   Genuine pairs (SD258): {target_matrix.sum()}")
    print(f"   Impostor pairs (TS1K): {num_queries * len(gallery_ts1k_templates)}")
    
    # Save results
    print(f"\n💾 Saving identification results...")
    results = {
        "scores_matrix": scores_matrix,
        "target_matrix": target_matrix,
        "query_ids": query_ids,
        "gallery_ids": gallery_ids,
        "metadata": {
            "num_queries": num_queries,
            "num_gallery_sd258": num_sd258_gallery,
            "num_gallery_ts1k": len(gallery_ts1k_templates),
            "num_gallery_total": num_gallery,
            "matches_found": matches_found
        }
    }
    with open(identification_results_path, "wb") as f:
        pkl.dump(results, f)
    print(f"   Saved to: {identification_results_path}")

## 6. CMC Curve (Cumulative Match Characteristic)

Evaluate identification performance at different ranks.

In [ ]:
# ========== RANKS TO DISPLAY ==========
# Define which ranks to show in results and annotations
DISPLAY_RANKS = [1, 10, 20, 30, 100, 1000]
# ========================================

def compute_cmc(score_matrix, target_matrix, max_rank=20):
    """
    Compute Cumulative Match Characteristic (CMC) curve.
    
    Args:
        score_matrix: Score matrix [num_queries, num_gallery]
        target_matrix: Ground truth matrix [num_queries, num_gallery]
        max_rank: Maximum rank to compute
        
    Returns:
        cmc: CMC values for ranks 1 to max_rank
    """
    num_queries = score_matrix.shape[0]
    
    # Get sorted indices (highest scores first)
    sorted_indices = np.argsort(score_matrix, axis=1)[:, ::-1]
    
    # Initialize CMC
    cmc = np.zeros(max_rank)
    
    # For each rank
    for rank in range(max_rank):
        # Get top-(rank+1) predictions
        top_k_indices = sorted_indices[:, :rank+1]
        
        # Check if any of top-k matches is genuine
        correct = 0
        for q_idx in range(num_queries):
            top_k = top_k_indices[q_idx]
            if np.any(target_matrix[q_idx, top_k]):
                correct += 1
        
        cmc[rank] = correct / num_queries
    
    return cmc

# Compute CMC
print("Computing CMC curve...")
cmc = compute_cmc(scores_matrix, target_matrix, max_rank=MAX_CMC_RANK)

# Print key metrics for configured ranks
print(f"\n📈 Identification Results:")
for rank in DISPLAY_RANKS:
    if rank <= len(cmc):
        print(f"   Rank-{rank:<3}: {cmc[rank-1]*100:.2f}%")

# Plot CMC curve
plt.figure(figsize=(12, 6))
ranks = np.arange(1, len(cmc) + 1)
plt.plot(ranks, cmc * 100, 'b-', linewidth=2, label='DMD++ with Background Gallery')
plt.xlabel('Rank', fontsize=12)
plt.ylabel('Identification Rate (%)', fontsize=12)
plt.title(f'CMC Curve - Gallery: {len(gallery_sd258_templates)} SD258 + {len(gallery_ts1k_templates)} TS1K', 
          fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3, linestyle='--')
plt.xlim([1, len(cmc)])
plt.ylim([0, 105])

# Set x-axis ticks
if len(cmc) <= 100:
    plt.xticks(np.arange(0, len(cmc) + 1, 10))
else:
    plt.xticks(np.arange(0, len(cmc) + 1, 25))

plt.legend(fontsize=11)

# Add value annotations for configured ranks
for rank in DISPLAY_RANKS:
    rank_idx = rank - 1  # Convert to 0-indexed
    if rank_idx < len(cmc):
        plt.annotate(f'{cmc[rank_idx]*100:.1f}%', 
                    xy=(rank, cmc[rank_idx] * 100),
                    xytext=(10, -10), textcoords='offset points',
                    fontsize=9, color='darkblue',
                    bbox=dict(boxstyle='round,pad=0.3', facecolor='yellow', alpha=0.3),
                    arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0', color='darkblue'))

plt.tight_layout()
plt.show()

print("\n✅ CMC curve plotted!")

## 7. Error Case Analysis

Visualize cases where Rank-1 prediction was incorrect.

In [19]:
# Helper function to get gallery image by index
def get_gallery_image(idx):
    """Get gallery image by index (handles both SD258 and TS1K)."""
    if idx < len(gallery_sd258):
        return gallery_sd258[idx]["orig"]
    else:
        ts1k_idx = idx - len(gallery_sd258)
        return gallery_ts1k[ts1k_idx]["orig"]

def get_gallery_id(idx):
    """Get gallery external ID by index (handles both SD258 and TS1K)."""
    if idx < len(gallery_sd258):
        return f"SD258: {noext(gallery_sd258[idx].external_id)}"
    else:
        ts1k_idx = idx - len(gallery_sd258)
        return f"TS1K: {noext(gallery_ts1k[ts1k_idx].external_id)}"

MAX_DISPLAY_ERR = 30

# Find error cases (where Rank-1 prediction is incorrect)
sorted_indices = np.argsort(scores_matrix, axis=1)[:, ::-1]
error_cases = []

for q_idx in range(len(query_templates)):
    # Get Rank-1 prediction
    rank1_idx = sorted_indices[q_idx, 0]
    
    # Check if Rank-1 is correct
    if not target_matrix[q_idx, rank1_idx]:
        # Find the correct match (should be at q_idx in SD258 gallery)
        correct_idx = np.where(target_matrix[q_idx])[0][0]
        
        # Find rank of correct match
        correct_rank = np.where(sorted_indices[q_idx] == correct_idx)[0][0] + 1
        
        error_cases.append({
            'query_idx': q_idx,
            'predicted_idx': rank1_idx,
            'correct_idx': correct_idx,
            'predicted_score': scores_matrix[q_idx, rank1_idx],
            'correct_score': scores_matrix[q_idx, correct_idx],
            'correct_rank': correct_rank
        })

print(f"🔴 Found {len(error_cases)} error cases (Rank-1 incorrect)")
print(f"   Error rate: {len(error_cases)/len(query_templates)*100:.2f}%")
print(f"   Displaying first {min(MAX_DISPLAY_ERR, len(error_cases))} cases\n")

# Display error cases
for i, err in enumerate(error_cases[:MAX_DISPLAY_ERR]):
    q_idx = err['query_idx']
    pred_idx = err['predicted_idx']
    correct_idx = err['correct_idx']
    
    # Get images
    query_img = queries[q_idx]["orig"]
    predicted_img = get_gallery_image(pred_idx)
    correct_img = get_gallery_image(correct_idx)
    
    # Plot
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    axes[0].imshow(query_img, cmap='gray')
    axes[0].set_title(f'Query #{q_idx}\n{noext(queries[q_idx].external_id)}', 
                     fontsize=11, fontweight='bold')
    axes[0].axis('off')
    
    axes[1].imshow(predicted_img, cmap='gray')
    axes[1].set_title(f'Predicted (Rank-1) #{pred_idx}\n{get_gallery_id(pred_idx)}\nScore: {err["predicted_score"]:.4f}', 
                     fontsize=11, color='red', fontweight='bold')
    axes[1].axis('off')
    
    axes[2].imshow(correct_img, cmap='gray')
    axes[2].set_title(f'Correct Match (Rank-{err["correct_rank"]}) #{correct_idx}\n{get_gallery_id(correct_idx)}\nScore: {err["correct_score"]:.4f}', 
                     fontsize=11, color='green', fontweight='bold')
    axes[2].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print(f"Error case {i+1}/{min(MAX_DISPLAY_ERR, len(error_cases))}: Query {q_idx} → Predicted {pred_idx} (should be {correct_idx} at rank {err['correct_rank']})")
    print(f"  Predicted score: {err['predicted_score']:.4f} | Correct score: {err['correct_score']:.4f}\n")

print("\n✅ Error analysis complete!")